In [1]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))
%load_ext autoreload
%autoreload 2
%aimport -

In [2]:
import numpy as np

from core import AutoGradBackend, TheanoBackend
import utils

# bk = AutoGradBackend()
bk = TheanoBackend()

class BinarySVMModelLoss(bk.ModelLoss):
    def __init__(self, n_features, n_steps=100):
        self.n_features = n_features
        self.n_steps = n_steps

        arg_dict = {
            'default_init_method': utils.inits.gaussian_init_with(mu=0, std=1),
            'weight_template': {
                'W': {'shape':(self.n_features, 1) },
                'b': {'shape':(1, ), 'init_method': np.random.rand },
            },
            'data_template': {
                'X': {'shape':('batch_size', self.n_features), 'dtype': 'float64' },
                'y': {'shape':('batch_size', ) , 'dtype': 'int64'},
            },
            'optimization_method': {'name': 'adam', 'learning_rate': 0.1},
        }
        super().__init__(**arg_dict)
    
    def loss(self, theta, data, const):
        A = bk.dot(data.X, theta.W) + theta.b # (batch_size, 1)
        loss_i = bk.maximum(0, 1 - A*data.y[:, bk.newaxis])
        return bk.sum(loss_i) + bk.sum(bk.abs(theta.W))
    
    @bk.export
    def score(self, theta, data, const):
        A = bk.dot(data.X, theta.W) + theta.b # (batch_size, 1)
        score = bk.sum( bk.isclose(bk.sign(A), data.y[:, bk.newaxis]))/A.shape[0]
        return score
    
    def step_callback(self, loss_val, theta, data, const, info):
        print(info['n_iter'], '%g' % loss_val, end='; ')
        print('%g' % self.compiled.loss(theta, data, const), end='! ')
        if info['n_iter'] == self.n_steps:
            print('\nFinal score:', self.compiled.score(theta, data, const))
        
from sklearn.datasets.samples_generator import make_classification

def test_svm():
    
    X, y_ = make_classification(n_samples=100, n_classes=2, n_features=20, 
                                n_informative=20, n_redundant=0, random_state=0)
    y = 2*y_-1
    model = BinarySVMModelLoss(n_features=20, n_steps=100)
    best_theta = model.fit({'X': X, 'y': y})
    
test_svm()

Using gpu device 2: Tesla K40c (CNMeM is disabled, cuDNN 4007)
/home/usman/anaconda2/envs/autograd_py3/lib/python3.5/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


Weight shapes are: ((1,), (20, 1)), n_total_params: 21
FAST_COMPILE
Building learning step function and gradient .... done
Building  score.. done
Building  loss.. done
0 447.01; 398.432! 1 398.432; 352.442! 2 352.442; 310.984! 3 310.983; 273.008! 4 273.008; 236.985! 5 236.985; 207.415! 6 207.415; 180.384! 7 180.384; 160.144! 8 160.144; 140.878! 9 140.878; 126.516! 10 126.516; 114.784! 11 114.784; 106.324! 12 106.324; 98.0942! 13 98.0942; 90.5866! 14 90.5866; 83.4901! 15 83.4901; 78.0625! 16 78.0625; 73.6473! 17 73.6473; 70.081! 18 70.081; 67.4297! 19 67.4297; 63.2208! 20 63.2208; 58.5304! 21 58.5304; 55.5578! 22 55.5578; 53.5849! 23 53.5849; 54.2713! 24 54.2713; 54.4525! 25 54.4525; 54.3468! 26 54.3468; 53.2108! 27 53.2108; 51.621! 28 51.621; 50.152! 29 50.152; 48.712! 30 48.712; 47.3085! 31 47.3085; 46.4827! 32 46.4827; 47.6202! 33 47.6202; 48.6459! 34 48.6459; 48.4803! 35 48.4803; 47.1977! 36 47.1977; 45.3195! 37 45.3195; 43.9108! 38 43.9108; 43.0199! 39 43.0199; 42.2059! 40 42.2059;